In [1]:
%load_ext autotime
import os, sys
import pandas as pd
pd.set_option('display.max_colwidth', 500)
sys.path.append('/root/data_disk/src')

from src.retriever import Retriever
from src.LLM_utils import LLM
from src.prompt import generate_prompt

# import prompt
# import importlib
# importlib.reload(prompt)

# from prompt import generate_prompt

time: 8.04 s (started: 2025-02-16 18:19:12 +08:00)


In [3]:
retriever = Retriever(embedder = 'BAAI/bge-m3',reranker = 'BAAI/bge-reranker-v2-m3',collection_name = 'sales_qa')
model_id = 'Qwen/Qwen2.5-3B-Instruct'
rewriter = LLM(model_id)

model_id = 'ep-20250213200344-crq6r'
gen_llm = LLM(model_id,api_key = os.getenv('ARK_API_KEY'),base_url = os.getenv('ARK_BASE_URL'))

2025-02-16 18:20:23,441 - modelscope - INFO - Creating symbolic link [/root/autodl-tmp/.cache/modelscope/Qwen/Qwen2.5-3B-Instruct].
2025-02-16 18:20:23,442 - modelscope - WARNING - Failed to create symbolic link /root/autodl-tmp/.cache/modelscope/Qwen/Qwen2.5-3B-Instruct for /root/autodl-tmp/.cache/modelscope/Qwen/Qwen2___5-3B-Instruct.
[TM][WARNING] [LlamaTritonModel] `max_context_token_num` is not set, default to 32768.


2025-02-16 18:20:24,249 - lmdeploy - WARNING - turbomind.py:217 - get 327 model params


[WARNING] gemm_config.in is not found; using default GEMM algo
time: 4.18 s (started: 2025-02-16 18:20:22 +08:00)


In [5]:
past_convs = []

time: 286 µs (started: 2025-02-16 18:20:27 +08:00)


In [6]:
query = """I need a BMW car"""
rewrite_prompt = generate_prompt('rewrite', query, past_convs, reranked_chunks=None)
rewritten_query = rewriter(rewrite_prompt)
past_convs.append("User: " + rewritten_query)

recall_docs, reranked_chunks = retriever.retrieve(rewritten_query)

2025-02-16 18:20:44,723 - lmdeploy - WARNING - async_engine.py:629 - GenerationConfig: GenerationConfig(n=1, max_new_tokens=512, do_sample=False, top_p=1.0, top_k=50, min_p=0.0, temperature=0.8, repetition_penalty=1.0, ignore_eos=False, random_seed=None, stop_words=None, bad_words=None, stop_token_ids=[151645], bad_token_ids=None, min_new_tokens=None, skip_special_tokens=True, spaces_between_special_tokens=True, logprobs=None, response_format=None, logits_processors=None, output_logits=None, output_last_hidden_state=None)
2025-02-16 18:20:44,724 - lmdeploy - WARNING - async_engine.py:630 - Since v0.6.0, lmdeploy add `do_sample` in GenerationConfig. It defaults to False, meaning greedy decoding. Please set `do_sample=True` if sampling  decoding is needed


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


time: 2.68 s (started: 2025-02-16 18:20:44 +08:00)


In [7]:
generation_prompt = generate_prompt('generate', query, past_convs, reranked_chunks)
response = gen_llm(generation_prompt)
past_convs.append("Assistant: " + response)

Submitting API request
time: 8 s (started: 2025-02-16 18:20:47 +08:00)


In [8]:
print("""\n""".join(past_convs))

User: I need a BMW car
Assistant: Based on your query, here are some BMW options that might suit your needs:

1. **BMW X3 M**  
   - A practical SUV with sports car-like performance. It’s fun to drive, has good visibility, and is comfortable for daily use. Fuel economy averages around 19 MPG, and it’s a great compromise if you’re used to sports cars or sedans.

2. **BMW 4 Series 430i**  
   - A stylish coupe with smooth and quiet performance, great for daily commutes. It offers excellent acceleration, a comfortable driver’s cockpit, and advanced tech features like voice command.

3. **BMW M2**  
   - A sporty coupe that delivers an ultimate driving experience. It’s fast, agile, and offers both manual and automatic transmission options. While the infotainment system has some drawbacks, it’s a fantastic choice for driving enthusiasts.

4. **BMW Z4**  
   - A fun-to-drive convertible roadster with quick acceleration and responsive handling. It’s a great value compared to competitors like 

In [2]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv

load_dotenv()
model = ChatCompletionsClient(
    endpoint=os.getenv('AZURE_ENDPOINT'),
    credential=AzureKeyCredential(os.getenv('AZURE_API_KEY')),
)

from azure.ai.inference.models import SystemMessage, UserMessage

response = model.complete(
    messages=[
        UserMessage(content='HI'),
    ],
    model="Llama-3.3-70B-Instruct"
)

In [3]:
print(response.choices[0].message.content)

Hello! How can I assist you today?
